## Patch extra attributes onto run object

In [1]:
# imports general modules, runs ipython magic commands
# change path in this notebook to point to repo locally
# n.b. sometimes need to run this cell twice to init the plotting paramters
%run ../setup_notebook.ipynb

In [2]:
import pickle
import run_functions as rf
import cacher
import utils_funcs as uf
import matplotlib.pyplot as plt
import os
import sys
import copy
from tqdm import tqdm
import math
from scipy import interpolate

oasis_path = os.path.expanduser('~/Documents/code/OASIS')
sys.path.append(oasis_path)

#from oasis.functions import deconvolve
from oasis import oasis_nan


In [3]:
mice_dicts = [
    
{'mouse_id': 'J048', 'run_numbers': [27,28,29,30,32], 'runs': []},
{'mouse_id': 'RL048', 'run_numbers': [23, 24, 25, 28, 29], 'runs': []},
{'mouse_id': 'J063', 'run_numbers': [8,9,10,11,12,13,14], 'runs': []},
{'mouse_id': 'J064', 'run_numbers': [10,11,12,13,14,15], 'runs': []},
{'mouse_id': 'J065', 'run_numbers': [10,11,12,13,14,15], 'runs': []},
{'mouse_id': 'RL070', 'run_numbers': [22,23,24,27,28,29], 'runs': []},
{'mouse_id': 'RL072', 'run_numbers': [19,20,21,24], 'runs': []},
    
]

In [4]:
# # Add extra attributes to run objects that I didn't consider at instantiation
# # time. Should be built into cacher for future runs
# for mouse_dict in mice_dicts:
#     for run_number in mouse_dict['run_numbers']:
        
#         pkl_path = os.path.join(path_to_vape, 'run_pkls', mouse_dict['mouse_id'], 
#                                 'run{}.pkl'.format(run_number))

#         with open(pkl_path, 'rb') as f:
#             run = pickle.load(f)

#         run.s2p_path = os.path.join(run.s2p_path, run.mouse_id)
#         try:
#             run = cacher.preprocess_flu(run)
#         except FileNotFoundError:
#             print(run.s2p_path)
#             raise

#         with open(pkl_path, 'wb') as f:
#             pickle.dump(run, f)
        
        
                

In [5]:
def fill_nan(A):
    '''
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

In [6]:
math.ceil(0.4*30)

12

In [7]:
def get_bad_frames(run, fs=30):    

    pre_frames = 2 #math.ceil(1*fs)
    post_frames = math.ceil(0.4*fs)

    paqio_frames = utils.tseries_finder(run.num_frames, run.frame_clock)
    trial_start = utils.get_spiral_start(run.x_galvo_uncaging, run.paq_rate*6)

    bad_frames = []
    trial_starts = []
    for i, start in enumerate(trial_start):
        
        frames, start_idx = utils.get_trial_frames(paqio_frames, start, 
                                                   pre_frames, post_frames, 
                                                   paq_rate=run.paq_rate, 
                                                   fs=fs)
        trial_starts.append(start_idx)
        bad_frames.append(frames)

    flattened = [bf for bf in bad_frames if bf is not None]
    flattened = [item for sublist in flattened for item in sublist]


    return trial_starts, bad_frames

In [8]:
def artifact_suppress(run, set_to=0, interpolate=False):
    
    if interpolate:
        set_to = np.nan

    if run.mouse_id == 'J048' or run.mouse_id == 'RL048':
        
        print('got a 5 here boi')

        fs = 5
        ssf = uf.stim_start_frame_mat(run.aligner.B_to_A(run.spiral_start), run.frames_ms)
        stim_start = np.min(ssf, 0)
        stim_start = stim_start[~np.isnan(stim_start)]

        pre_frames = 2 #math.ceil(1*fs)
        post_frames = 3 #math.ceil(1*fs)

        bad_frames = []

        for stim in stim_start:
            bad_frames.append(np.arange(stim-pre_frames, stim+post_frames))

        bad_frames = np.concatenate(bad_frames).astype('int')

        flu = copy.deepcopy(run.flu)
        flu[:, bad_frames] = set_to
        
        if interpolate:
            flu = np.array([fill_nan(f) for f in flu])
            assert flu.shape == run.flu.shape
            
    else:
        print('oooh 30 here we go')
        fs = 30
        trial_starts, bad = get_bad_frames(run)
        bad = [b for b in bad if b is not None]
        bad = np.concatenate(bad)

        flu = copy.deepcopy(run.flu)
        flu[:, bad] = set_to
        
        if interpolate:
            flu = np.array([fill_nan(f) for f in flu])
            assert flu.shape == run.flu.shape
    
    return flu.astype('float64')


In [9]:
# Add extra attributes to run objects that I didn't consider at instantiation
# time. Should be built into cacher for future runs
for mouse_dict in mice_dicts:
    for run_number in mouse_dict['run_numbers']:
        
        
        pkl_path = os.path.join(path_to_vape, 'run_pkls', mouse_dict['mouse_id'], 
                                'run{}.pkl'.format(run_number))

        with open(pkl_path, 'rb') as f:
            run = pickle.load(f)
        
        if not hasattr(run, 'flu'):
            continue
        
        #flu = run.flu.astype('float64')
        flu = artifact_suppress(run, set_to=np.nan, interpolate=False)
        denoised_flu = np.empty_like(flu)
        spks = np.empty_like(flu)
        
        for idx, cell in enumerate(tqdm(flu)):
            #c, s, b, g, lam = deconvolve(cell, penalty=0)
            assert np.isnan(cell).any()
            c, s = oasis_nan.oasisAR1(cell, .95, s_min=.5)
            c[np.isnan(cell)] = np.nan
            s[np.isnan(cell)] = np.nan
            denoised_flu[idx, :] = c
            spks[idx, :] = s
        
        run.spks = spks
        run.denoised_flu = denoised_flu
        
            
        with open(pkl_path, 'wb') as f:
            pickle.dump(run, f)
        

got a 5 here boi


100%|██████████| 2101/2101 [00:07<00:00, 266.63it/s]


got a 5 here boi


100%|██████████| 2194/2194 [00:09<00:00, 236.17it/s]


got a 5 here boi


...

100%|██████████| 201/201 [00:08<00:00, 22.39it/s]


oooh 30 here we go


100%|██████████| 625/625 [00:22<00:00, 28.10it/s]


oooh 30 here we go


100%|██████████| 718/718 [00:19<00:00, 36.17it/s]


In [10]:
flu.shape

(718, 113815)

In [11]:
for i in range(flu.shape[0]):
    c = flu[i,:]
    print(np.isnan(c).any())

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [12]:

x = np.nanmean(np.abs(run.spks), 1) < 1

for i in x:
    print(i)

AttributeError: 'BlimpImport' object has no attribute 'spks'

In [ ]:
run.flu.shape

In [ ]:
n_cells = 5

plt.figure(figsize=(30,10))
flu = run.flu
for i in range(n_cells):
    plt.plot(flu[i,:] + 2*i)

In [ ]:
run.paqio_frames

In [ ]:
run.spiral_start

In [ ]:
run.__dict__.keys()

In [ ]:
run.spks